In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

import sys

# setting path
sys.path.append('..')

from evaluation import get_time_step_fleet
from utils import load_solution

In [2]:
solution = load_solution("../data/solution_example.json")
df = get_time_step_fleet(solution=solution, ts=1)

In [3]:
df.columns

Index(['datacenter_id', 'server_generation', 'server_id', 'action'], dtype='object')

In [4]:
from utils import load_problem_data
demand, datacenters, servers, selling_prices = load_problem_data('../data')

In [5]:
demand_selected = demand[demand['time_step'] == 1]
demand_selected

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3
0,1,high,4000,0,0,0,30,0,0
168,1,medium,6000,0,0,0,10,0,0
336,1,low,10000,0,0,0,10,0,0


In [6]:
datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


In [7]:
merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')

# Calculate Total Revenue (assuming selling price represents revenue per time unit over life expectancy)
merged_servers_df['total_revenue'] = merged_servers_df['capacity'] * merged_servers_df['selling_price'] * merged_servers_df['life_expectancy']

# Calculate Total Costs
merged_servers_df['total_cost'] = (
    merged_servers_df['purchase_price'] +
    (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
    merged_servers_df['cost_of_moving']
)

# Calculate Profitability
merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']

# Sort by Profitability
most_profitable_servers = merged_servers_df.sort_values(by='profitability', ascending=False)

most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
20,GPU.S3,high,2700.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,2073600.0,456680,1616920.0
13,GPU.S3,medium,2450.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1881600.0,456680,1424920.0
6,GPU.S3,low,2150.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1651200.0,456680,1194520.0
19,GPU.S2,high,2000.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1536000.0,399720,1136280.0
18,GPU.S1,high,1875.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1440000.0,342760,1097240.0
12,GPU.S2,medium,1800.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1382400.0,399720,982680.0
11,GPU.S1,medium,1680.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1290240.0,342760,947480.0
5,GPU.S2,low,1600.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1228800.0,399720,829080.0
4,GPU.S1,low,1500.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1152000.0,342760,809240.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,460800.0,63608,397192.0


In [8]:
most_profitable_servers.to_csv("most_profitable_servers.csv")

In [9]:
most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
20,GPU.S3,high,2700.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,2073600.0,456680,1616920.0
13,GPU.S3,medium,2450.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1881600.0,456680,1424920.0
6,GPU.S3,low,2150.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1651200.0,456680,1194520.0
19,GPU.S2,high,2000.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1536000.0,399720,1136280.0
18,GPU.S1,high,1875.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1440000.0,342760,1097240.0
12,GPU.S2,medium,1800.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1382400.0,399720,982680.0
11,GPU.S1,medium,1680.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1290240.0,342760,947480.0
5,GPU.S2,low,1600.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1228800.0,399720,829080.0
4,GPU.S1,low,1500.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1152000.0,342760,809240.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,460800.0,63608,397192.0


In [10]:
demand['CPU.S1'].max()/60

11654.4

In [11]:
import uuid


def generate_random_fleet(datacenters=datacenters, servers=servers, num_servers=100, max_time_step=168):
    # Create lists of valid datacenter IDs and server generations
    datacenter_ids = datacenters['datacenter_id'].tolist()
    server_generations = servers['server_generation'].tolist()

    # Generate random data
    random_fleet = pd.DataFrame({
        'datacenter_id': np.random.choice(datacenter_ids, num_servers),
        'server_generation': np.random.choice(server_generations, num_servers),
        'server_id': [str(uuid.uuid4()) for _ in range(num_servers)],
        'time_step_of_purchase': np.random.randint(1, max_time_step + 1, num_servers)
    })

    return random_fleet

In [12]:
fleet = generate_random_fleet()
fleet

,datacenter_id,server_generation,server_id,time_step_of_purchase
0,DC4,CPU.S3,f1ce93bb-2fa5-4952-92e4-c86ed54e1600,161
1,DC3,CPU.S3,feb1ddef-f492-422e-bf02-fb3ca5cdb8c8,35
2,DC2,CPU.S3,295ee20c-b5d8-4aa7-8fbc-7c8060a15468,135
3,DC2,CPU.S3,d6b85277-4cdc-473f-ab93-7b1bd7c7b3f0,47
4,DC1,CPU.S2,aa5ce5d7-5322-48ed-9fcd-628315671de9,21
...,...,...,...,...
95,DC2,CPU.S1,1e90fc36-bbbd-47a2-bf19-4efc4ed7e49c,41
96,DC4,CPU.S1,dd3995cd-1fc4-4667-bdd6-d1f1fe15475f,24
97,DC3,GPU.S2,6e2f1bd2-40c1-4438-86b5-af69f4443218,105
98,DC2,CPU.S3,4b76d192-4079-4d4d-9e2c-c3a5a5760caf,74


In [13]:
server_counts = fleet.groupby(['datacenter_id', 'server_generation']).size().reset_index(name="count")
server_counts

,datacenter_id,server_generation,count
0,DC1,CPU.S1,2
1,DC1,CPU.S2,8
2,DC1,CPU.S3,4
3,DC1,CPU.S4,7
4,DC1,GPU.S1,6
5,DC1,GPU.S2,1
6,DC1,GPU.S3,1
7,DC2,CPU.S1,6
8,DC2,CPU.S2,1
9,DC2,CPU.S3,12


In [14]:
fleet.loc[fleet["datacenter_id"] == "DC4"]

,datacenter_id,server_generation,server_id,time_step_of_purchase
0,DC4,CPU.S3,f1ce93bb-2fa5-4952-92e4-c86ed54e1600,161
5,DC4,CPU.S3,71656635-a231-493f-b38f-0148c2598b65,72
8,DC4,GPU.S1,28156c6f-11bf-4d8b-9f56-577e65be9510,27
21,DC4,CPU.S3,1c3da510-6120-4f62-bff6-43c65b3d351b,4
23,DC4,GPU.S1,e9fc88c9-8390-4edb-b998-35779c9159d5,151
37,DC4,CPU.S3,f68ac7fa-5771-41ab-8a94-3026303cdf17,82
39,DC4,GPU.S2,d1486e23-7634-44b5-b025-61c72fb8245a,100
41,DC4,CPU.S4,ba084c10-7143-4681-932c-0fd819e9740a,96
45,DC4,GPU.S1,30ee3ce1-4e2a-43c4-a632-3f061c047131,16
47,DC4,GPU.S3,a1e87424-d90c-493e-8601-90476e01408e,155


In [15]:
def get_server_ages(fleet, time_step):
    return fleet['time_step_of_purchase'].apply(lambda x: time_step - x)

In [16]:
get_server_ages(fleet, 168)

0       7
1     133
2      33
3     121
4     147
     ... 
95    127
96    144
97     63
98     94
99    153
Name: time_step_of_purchase, Length: 100, dtype: int64

In [17]:
# import pandas as pd
# from typing import List, Dict
# import uuid

# def load_and_preprocess_data():
#     # Load data
#     datacenters = pd.read_csv('../data/datacenters.csv')
#     servers = pd.read_csv('../data/servers.csv')
#     selling_prices = pd.read_csv('../data/selling_prices.csv')
#     demand = pd.read_csv('../data/demand.csv')

#     # Sort datacenters by cost_of_energy
#     datacenters = datacenters.sort_values('cost_of_energy')

#     return datacenters, servers, selling_prices, demand

# def calculate_server_profitability(servers: pd.DataFrame, selling_prices: pd.DataFrame) -> pd.DataFrame:
#     merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')
    
#     merged_servers_df['total_revenue'] = (
#         merged_servers_df['capacity'] * 
#         merged_servers_df['selling_price'] * 
#         merged_servers_df['life_expectancy']
#     )
    
#     merged_servers_df['total_cost'] = (
#         merged_servers_df['purchase_price'] +
#         (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
#         merged_servers_df['cost_of_moving']
#     )
    
#     merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']
    
#     return merged_servers_df.sort_values(by='profitability', ascending=False)

# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate total demand across all server types and latency sensitivities
#     total_demand = sum(current_demand[col] for col in current_demand.index if col not in ['time_step', 'latency_sensitivity'])
    
#     # Calculate current capacity
#     current_capacity = sum(server['capacity'] for server in current_fleet)
    
#     if total_demand > current_capacity:
#         # We need to add more servers
#         servers_to_add = total_demand - current_capacity
        
#         for _, datacenter in datacenters.iterrows():
#             if servers_to_add <= 0:
#                 break
            
#             available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
            
#             for _, server in servers.iterrows():
#                 if servers_to_add <= 0 or available_slots < server['slots_size']:
#                     break
                
#                 release_start, release_end = map(int, server['release_time'][1:-1].split(','))
#                 if release_start <= time_step <= release_end:
#                     # Server is available for purchase
#                     num_servers = min(servers_to_add // server['capacity'], available_slots // server['slots_size'])
                    
#                     for _ in range(num_servers):
#                         new_server = {
#                             'time_step': time_step,
#                             'datacenter_id': datacenter['datacenter_id'],
#                             'server_generation': server['server_generation'],
#                             'server_id': str(uuid.uuid4()),
#                             'action': 'buy',
#                             'capacity': server['capacity'],
#                             'slots_size': server['slots_size']
#                         }
#                         actions.append(new_server)
#                         current_fleet.append(new_server)
                        
#                         servers_to_add -= server['capacity']
#                         available_slots -= server['slots_size']
    
#     elif total_demand < current_capacity:
#         # We need to remove servers
#         servers_to_remove = current_capacity - total_demand
        
#         # Sort current fleet by datacenter cost (descending) and server age (descending)
#         current_fleet.sort(key=lambda x: (
#             -datacenters[datacenters['datacenter_id'] == x['datacenter_id']]['cost_of_energy'].iloc[0],
#             -x['time_step']
#         ))
        
#         for server in current_fleet:
#             if servers_to_remove <= 0:
#                 break
            
#             server_capacity = servers[servers['server_generation'] == server['server_generation']]['capacity'].iloc[0]
            
#             if servers_to_remove >= server_capacity:
#                 actions.append({
#                     'time_step': time_step,
#                     'datacenter_id': server['datacenter_id'],
#                     'server_generation': server['server_generation'],
#                     'server_id': server['server_id'],
#                     'action': 'dismiss'
#                 })
#                 current_fleet.remove(server)
#                 servers_to_remove -= server_capacity
    
#     return actions, current_fleet

# def run_simulation(datacenters: pd.DataFrame, 
#                    servers: pd.DataFrame, 
#                    selling_prices: pd.DataFrame, 
#                    demand: pd.DataFrame) -> List[Dict]:
#     profitable_servers = calculate_server_profitability(servers, selling_prices)
#     servers = servers.merge(profitable_servers[['server_generation', 'profitability']], on='server_generation')
#     servers = servers.sort_values('profitability', ascending=False)
    
#     all_actions = []
#     current_fleet = []
    
#     for time_step in range(1, 169):  # 168 time steps
#         actions, current_fleet = allocate_servers(time_step, demand, datacenters, servers, current_fleet)
#         all_actions.extend(actions)
    
#     return all_actions

# def main():
#     datacenters, servers, selling_prices, demand = load_and_preprocess_data()
#     solution = run_simulation(datacenters, servers, selling_prices, demand)
    
#     # Convert solution to DataFrame and save as JSON
#     solution_df = pd.DataFrame(solution)
#     solution_df.to_json('solution.json', orient='records', indent=2)

# if __name__ == "__main__":
#     main()

In [18]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict
# import uuid

# def load_and_preprocess_data():
#     # Load data
#     datacenters = pd.read_csv('../data/datacenters.csv')
#     servers = pd.read_csv('../data/servers.csv')
#     selling_prices = pd.read_csv('../data/selling_prices.csv')
#     demand = pd.read_csv('../data/demand.csv')

#     # Sort datacenters by cost_of_energy
#     datacenters = datacenters.sort_values('cost_of_energy')

#     return datacenters, servers, selling_prices, demand

# def calculate_server_profitability(servers: pd.DataFrame, selling_prices: pd.DataFrame) -> pd.DataFrame:
#     merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')
    
#     merged_servers_df['total_revenue'] = (
#         merged_servers_df['capacity'] * 
#         merged_servers_df['selling_price'] * 
#         merged_servers_df['life_expectancy']
#     )
    
#     merged_servers_df['total_cost'] = (
#         merged_servers_df['purchase_price'] +
#         (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
#         merged_servers_df['cost_of_moving']
#     )
    
#     merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']
    
#     return merged_servers_df.sort_values(by='profitability', ascending=False)

# def calculate_demand(current_demand: pd.Series, servers: pd.DataFrame) -> Dict[str, int]:
#     demand_dict = {}
#     for server_type in ['CPU', 'GPU']:
#         for latency in ['high', 'medium', 'low']:
#             server_gens = servers[servers['server_type'] == server_type]['server_generation'].tolist()
#             demand = sum(current_demand[gen] for gen in server_gens if gen in current_demand.index)
#             capacity = servers[servers['server_type'] == server_type]['capacity'].iloc[0]
#             demand_dict[f"{server_type}_{latency}"] = int(np.ceil(demand / capacity))
#     return demand_dict

# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = {f"{server['server_type']}_{server['latency_sensitivity']}": 0 for server in current_fleet}
#     for server in current_fleet:
#         current_capacity[f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         servers_to_add = max(0, required_servers - current_capacity[server_type_latency])
#         servers_to_remove = max(0, current_capacity[server_type_latency] - required_servers)
        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 for _, server in servers[(servers['server_type'] == server_type) & (servers['profitability'] == servers['profitability'].max())].iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     release_start, release_end = map(int, server['release_time'][1:-1].split(','))
#                     if release_start <= time_step <= release_end:
#                         # Server is available for purchase
#                         num_servers = min(servers_to_add, available_slots // server['slots_size'])
                        
#                         for _ in range(num_servers):
#                             new_server = {
#                                 'time_step': time_step,
#                                 'datacenter_id': datacenter['datacenter_id'],
#                                 'server_generation': server['server_generation'],
#                                 'server_id': str(uuid.uuid4()),
#                                 'action': 'buy',
#                                 'capacity': server['capacity'],
#                                 'slots_size': server['slots_size'],
#                                 'server_type': server_type,
#                                 'latency_sensitivity': latency
#                             }
#                             actions.append(new_server)
#                             current_fleet.append(new_server)
                            
#                             servers_to_add -= 1
#                             available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers
#             servers_to_remove_list = []
#             for server in sorted(current_fleet, key=lambda x: (x['time_step'], -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0])):
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

# def run_simulation(datacenters: pd.DataFrame, 
#                    servers: pd.DataFrame, 
#                    selling_prices: pd.DataFrame, 
#                    demand: pd.DataFrame) -> List[Dict]:
#     profitable_servers = calculate_server_profitability(servers, selling_prices)
#     servers = servers.merge(profitable_servers[['server_generation', 'profitability']], on='server_generation')
#     servers = servers.sort_values('profitability', ascending=False)
    
#     all_actions = []
#     current_fleet = []
    
#     for time_step in range(1, 169):  # 168 time steps
#         actions, current_fleet = allocate_servers(time_step, demand, datacenters, servers, current_fleet)
#         all_actions.extend(actions)
    
#     return all_actions

# def main():
#     datacenters, servers, selling_prices, demand = load_and_preprocess_data()
#     solution = run_simulation(datacenters, servers, selling_prices, demand)
    
#     # Convert solution to DataFrame and save as JSON
#     solution_df = pd.DataFrame(solution)
#     solution_df.to_json('solution.json', orient='records', indent=2)

# if __name__ == "__main__":
#     main()

In [19]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import uuid

def load_and_preprocess_data():
    # Load data
    datacenters = pd.read_csv('../data/datacenters.csv')
    servers = pd.read_csv('../data/servers.csv')
    selling_prices = pd.read_csv('../data/selling_prices.csv')
    demand = pd.read_csv('../data/demand.csv')

    # Sort datacenters by cost_of_energy
    datacenters = datacenters.sort_values('cost_of_energy')

    return datacenters, servers, selling_prices, demand

def calculate_server_profitability(servers: pd.DataFrame, selling_prices: pd.DataFrame) -> pd.DataFrame:
    merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')
    
    merged_servers_df['total_revenue'] = (
        merged_servers_df['capacity'] * 
        merged_servers_df['selling_price'] * 
        merged_servers_df['life_expectancy']
    )
    
    merged_servers_df['total_cost'] = (
        merged_servers_df['purchase_price'] +
        (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
        merged_servers_df['cost_of_moving']
    )
    
    merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']
    
    return merged_servers_df.sort_values(by='profitability', ascending=False)

def calculate_demand(current_demand: pd.Series, servers: pd.DataFrame) -> Dict[str, int]:
    demand_dict = {}
    for server_type in ['CPU', 'GPU']:
        for latency in ['high', 'medium', 'low']:
            server_gens = servers[servers['server_type'] == server_type]['server_generation'].tolist()
            demand = sum(current_demand.get(gen, 0) for gen in server_gens)
            capacity = servers[servers['server_type'] == server_type]['capacity'].iloc[0]
            demand_dict[f"{server_type}_{latency}"] = int(np.ceil(demand / capacity))
    return demand_dict

def initialize_current_capacity():
    return {f"{server_type}_{latency}": 0 
            for server_type in ['CPU', 'GPU'] 
            for latency in ['high', 'medium', 'low']}


In [20]:

# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = initialize_current_capacity()
#     for server in current_fleet:
#         current_capacity[f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         servers_to_add = max(0, required_servers - current_capacity[server_type_latency])
#         servers_to_remove = max(0, current_capacity[server_type_latency] - required_servers)
        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 for _, server in servers[(servers['server_type'] == server_type) & (servers['profitability'] == servers['profitability'].max())].iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     release_start, release_end = map(int, server['release_time'][1:-1].split(','))
#                     if release_start <= time_step <= release_end:
#                         # Server is available for purchase
#                         num_servers = min(servers_to_add, available_slots // server['slots_size'])
                        
#                         for _ in range(num_servers):
#                             new_server = {
#                                 'time_step': time_step,
#                                 'datacenter_id': datacenter['datacenter_id'],
#                                 'server_generation': server['server_generation'],
#                                 'server_id': str(uuid.uuid4()),
#                                 'action': 'buy',
#                                 'capacity': server['capacity'],
#                                 'slots_size': server['slots_size'],
#                                 'server_type': server_type,
#                                 'latency_sensitivity': latency
#                             }
#                             actions.append(new_server)
#                             current_fleet.append(new_server)
                            
#                             servers_to_add -= 1
#                             available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers
#             servers_to_remove_list = []
#             for server in sorted(current_fleet, key=lambda x: (x['time_step'], -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0])):
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

In [21]:
# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = initialize_current_capacity()
#     for server in current_fleet:
#         current_capacity[f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         servers_to_add = max(0, required_servers - current_capacity[server_type_latency])
#         servers_to_remove = max(0, current_capacity[server_type_latency] - required_servers)
        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 # Filter servers by type and availability at current time step
#                 available_servers = servers[(servers['server_type'] == server_type) & 
#                                             (servers['release_time'].apply(lambda x: eval(x)[0] <= time_step <= eval(x)[1]))]
                
#                 # Sort available servers by profitability
#                 available_servers = available_servers.sort_values('profitability', ascending=False)
                
#                 for _, server in available_servers.iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     # Server is available for purchase
#                     num_servers = min(servers_to_add, available_slots // server['slots_size'])
                    
#                     for _ in range(num_servers):
#                         # Create a base server dictionary for actions
#                         new_server_actions = {
#                             'time_step': time_step,
#                             'datacenter_id': datacenter['datacenter_id'],
#                             'server_generation': server['server_generation'],
#                             'server_id': str(uuid.uuid4()),
#                             'action': 'buy'
#                         }

#                         # Add the simplified version to actions
#                         actions.append(new_server_actions)

#                         # Create a more detailed server dictionary for current_fleet
#                         new_server_fleet = {
#                             **new_server_actions,  # start with the base attributes from new_server_actions
#                             'capacity': server['capacity'],
#                             'slots_size': server['slots_size'],
#                             'server_type': server_type,
#                             'latency_sensitivity': latency
#                         }

#                         # Add the detailed version to current_fleet
#                         current_fleet.append(new_server_fleet)
                        
#                         servers_to_add -= 1
#                         available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers (this part remains the same)
#             servers_to_remove_list = []
#             for server in sorted(current_fleet, key=lambda x: (x['time_step'], -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0])):
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

In [22]:
# def allocate_servers(time_step: int, 
#                      demand: pd.DataFrame, 
#                      datacenters: pd.DataFrame, 
#                      servers: pd.DataFrame, 
#                      current_fleet: List[Dict]) -> List[Dict]:
#     actions = []
    
#     # Get demand for the current time step
#     current_demand = demand[demand['time_step'] == time_step].iloc[0]
    
#     # Calculate demand for each server type and latency sensitivity
#     demand_dict = calculate_demand(current_demand, servers)
    
#     # Calculate current capacity
#     current_capacity = {datacenter['datacenter_id']: initialize_current_capacity() for _, datacenter in datacenters.iterrows()}
#     for server in current_fleet:
#         current_capacity[server['datacenter_id']][f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
#     for server_type_latency, required_servers in demand_dict.items():
#         server_type, latency = server_type_latency.split('_')
#         total_current_capacity = sum(capacity[server_type_latency] for capacity in current_capacity.values())
#         servers_to_add = max(0, required_servers - total_current_capacity)
#         servers_to_remove = max(0, total_current_capacity - required_servers)

        
#         if servers_to_add > 0:
#             # Add servers
#             for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
#                 if servers_to_add <= 0:
#                     break
                
#                 available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
#                 # Filter servers by type and availability at current time step
#                 available_servers = servers[(servers['server_type'] == server_type) & 
#                                             (servers['release_time'].apply(lambda x: eval(x)[0] <= time_step <= eval(x)[1]))]
                
#                 # Sort available servers by profitability
#                 available_servers = available_servers.sort_values('profitability', ascending=False)
                
#                 for _, server in available_servers.iterrows():
#                     if servers_to_add <= 0 or available_slots < server['slots_size']:
#                         break
                    
#                     # Server is available for purchase
#                     num_servers = min(servers_to_add, available_slots // server['slots_size'])
                    
#                     for _ in range(num_servers):
#                         # Create a base server dictionary for actions
#                         new_server_actions = {
#                             'time_step': time_step,
#                             'datacenter_id': datacenter['datacenter_id'],
#                             'server_generation': server['server_generation'],
#                             'server_id': str(uuid.uuid4()),
#                             'action': 'buy'
#                         }

#                         # Add the simplified version to actions
#                         actions.append(new_server_actions)

#                         # Create a more detailed server dictionary for current_fleet
#                         new_server_fleet = {
#                             **new_server_actions,  # start with the base attributes from new_server_actions
#                             'capacity': server['capacity'],
#                             'slots_size': server['slots_size'],
#                             'server_type': server_type,
#                             'latency_sensitivity': latency
#                         }

#                         # Add the detailed version to current_fleet
#                         current_fleet.append(new_server_fleet)
#                         current_capacity[datacenter['datacenter_id']][server_type_latency] += 1
                        
#                         servers_to_add -= 1
#                         available_slots -= server['slots_size']
        
#         elif servers_to_remove > 0:
#             # Remove servers
#             servers_to_remove_list = []
#             # Sort current fleet by time_step (oldest first) and then by inverse profitability
#             sorted_fleet = sorted(
#                 current_fleet,
#                 key=lambda x: (
#                     x['time_step'],
#                     -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0]
#                 )
#             )
#             for server in sorted_fleet:
#                 if servers_to_remove <= 0:
#                     break
                
#                 if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
#                     actions.append({
#                         'time_step': time_step,
#                         'datacenter_id': server['datacenter_id'],
#                         'server_generation': server['server_generation'],
#                         'server_id': server['server_id'],
#                         'action': 'dismiss'
#                     })
#                     servers_to_remove_list.append(server)
#                     servers_to_remove -= 1
#                     current_capacity[server['datacenter_id']][server_type_latency] -= 1
            
#             for server in servers_to_remove_list:
#                 current_fleet.remove(server)
    
#     return actions, current_fleet

In [23]:
def check_and_dismiss_old_servers(time_step: int, current_fleet: List[Dict], life_expectancy: int = 96) -> Tuple[List[Dict], List[Dict]]:
    actions = []
    servers_to_remove = []
    
    for server in current_fleet:
        server_age = time_step - server['time_step'] + 1
        if server_age == life_expectancy - 1:  # Dismiss at 95th time step
            actions.append({
                'time_step': time_step,
                'datacenter_id': server['datacenter_id'],
                'server_generation': server['server_generation'],
                'server_id': server['server_id'],
                'action': 'dismiss'
            })
            servers_to_remove.append(server)
    
    for server in servers_to_remove:
        current_fleet.remove(server)
    
    return actions, current_fleet

In [24]:
def allocate_servers(time_step: int, 
                     demand: pd.DataFrame, 
                     datacenters: pd.DataFrame, 
                     servers: pd.DataFrame, 
                     current_fleet: List[Dict]) -> List[Dict]:
    # First, check and dismiss old servers
    old_server_actions, current_fleet = check_and_dismiss_old_servers(time_step, current_fleet)
    
    actions = old_server_actions  # Start with actions from dismissing old servers
    
    # Get demand for the current time step 
    # TODO: MISTAKE IT PICKS ONLY THE FIRST RECORD WHICH IS FOR HIGH DEMAND
    current_demand = demand[demand['time_step'] == time_step].iloc[0] 
    
    # Calculate demand for each server type and latency sensitivity
    demand_dict = calculate_demand(current_demand, servers)
    
    # Calculate current capacity
    current_capacity = {datacenter['datacenter_id']: initialize_current_capacity() for _, datacenter in datacenters.iterrows()}
    for server in current_fleet:
        current_capacity[server['datacenter_id']][f"{server['server_type']}_{server['latency_sensitivity']}"] += 1
    
    for server_type_latency, required_servers in demand_dict.items():
        server_type, latency = server_type_latency.split('_')
        total_current_capacity = sum(capacity[server_type_latency] for capacity in current_capacity.values())
        servers_to_add = max(0, required_servers - total_current_capacity)
        servers_to_remove = max(0, total_current_capacity - required_servers)
        
        if servers_to_add > 0:
            # Add servers
            for _, datacenter in datacenters[datacenters['latency_sensitivity'] == latency].iterrows():
                if servers_to_add <= 0:
                    break
                
                available_slots = datacenter['slots_capacity'] - sum(server['slots_size'] for server in current_fleet if server['datacenter_id'] == datacenter['datacenter_id'])
                
                # Filter servers by type and availability at current time step
                available_servers = servers[(servers['server_type'] == server_type) & 
                                            (servers['release_time'].apply(lambda x: eval(x)[0] <= time_step <= eval(x)[1]))]
                
                # Sort available servers by profitability
                available_servers = available_servers.sort_values('profitability', ascending=False)
                
                for _, server in available_servers.iterrows():
                    if servers_to_add <= 0 or available_slots < server['slots_size']:
                        break
                    
                    # Server is available for purchase
                    num_servers = min(servers_to_add, available_slots // server['slots_size'])
                    
                    for _ in range(num_servers):
                        # Create a base server dictionary for actions
                        new_server_actions = {
                            'time_step': time_step,
                            'datacenter_id': datacenter['datacenter_id'],
                            'server_generation': server['server_generation'],
                            'server_id': str(uuid.uuid4()),
                            'action': 'buy'
                        }

                        # Add the simplified version to actions
                        actions.append(new_server_actions)

                        # Create a more detailed server dictionary for current_fleet
                        new_server_fleet = {
                            **new_server_actions,  # start with the base attributes from new_server_actions
                            'capacity': server['capacity'],
                            'slots_size': server['slots_size'],
                            'server_type': server_type,
                            'latency_sensitivity': latency
                        }

                        # Add the detailed version to current_fleet
                        current_fleet.append(new_server_fleet)
                        current_capacity[datacenter['datacenter_id']][server_type_latency] += 1
                        
                        servers_to_add -= 1
                        available_slots -= server['slots_size']
        
        elif servers_to_remove > 0:
            # Remove servers
            servers_to_remove_list = []
            # Sort current fleet by time_step (oldest first) and then by inverse profitability
            sorted_fleet = sorted(
                current_fleet,
                key=lambda x: (
                    x['time_step'],
                    -servers[servers['server_generation'] == x['server_generation']]['profitability'].iloc[0]
                )
            )
            for server in sorted_fleet:
                if servers_to_remove <= 0:
                    break
                
                if server['server_type'] == server_type and server['latency_sensitivity'] == latency:
                    actions.append({
                        'time_step': time_step,
                        'datacenter_id': server['datacenter_id'],
                        'server_generation': server['server_generation'],
                        'server_id': server['server_id'],
                        'action': 'dismiss'
                    })
                    servers_to_remove_list.append(server)
                    servers_to_remove -= 1
                    current_capacity[server['datacenter_id']][server_type_latency] -= 1
            
            for server in servers_to_remove_list:
                current_fleet.remove(server)
    
    return actions, current_fleet

In [25]:
def run_simulation(datacenters: pd.DataFrame, 
                   servers: pd.DataFrame, 
                   selling_prices: pd.DataFrame, 
                   demand: pd.DataFrame) -> List[Dict]:
    profitable_servers = calculate_server_profitability(servers, selling_prices)
    servers = servers.merge(profitable_servers[['server_generation', 'profitability']], on='server_generation')
    servers = servers.sort_values('profitability', ascending=False)
    
    all_actions = []
    current_fleet = []
    
    for time_step in range(1, 169):  # 168 time steps
        actions, current_fleet = allocate_servers(time_step, demand, datacenters, servers, current_fleet)
        all_actions.extend(actions)
    
    return all_actions

In [26]:
# def main():
#     datacenters, servers, selling_prices, demand = load_and_preprocess_data()
#     solution = run_simulation(datacenters, servers, selling_prices, demand)
    
#     # Convert solution to DataFrame and save as JSON
#     solution_df = pd.DataFrame(solution)
#     solution_df.to_json('solution.json', orient='records', indent=2)

# if __name__ == "__main__":
#     main()

In [27]:
def calculate_demand(current_demand: pd.Series, servers: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the demand for each server generation and latency sensitivity.
    
    Args:
        current_demand (pd.DataFrame): Current demand for each server generation.   
        servers (pd.DataFrame): DataFrame containing server information.
    
    Returns:
        pd.DataFrame: A DataFrame with the following columns:
        - server_generation (str): The generation of the server.
        - latency (str): Latency sensitivity (high, medium, or low).
        - demand (int): Calculated demand for the server generation and latency.
    """
    # Create a DataFrame from the current_demand Series
    demand_df = current_demand.reset_index()
    demand_df.columns = ['server_generation', 'demand']

    # Create a DataFrame with all combinations of server generations and latencies
    latencies = pd.DataFrame({'latency': ['high', 'medium', 'low']})
    demand_df = demand_df.merge(latencies, how='cross')

    # Ensure demand is an integer
    demand_df['demand'] = demand_df['demand'].astype(int)

    return demand_df[['server_generation', 'latency', 'demand']]

In [28]:
demand[demand['time_step'] == 1].iloc[0]

time_step                 1
latency_sensitivity    high
CPU.S1                 4000
CPU.S2                    0
CPU.S3                    0
CPU.S4                    0
GPU.S1                   30
GPU.S2                    0
GPU.S3                    0
Name: 0, dtype: object

In [29]:
def calculate_servers_needed(demand_df: pd.DataFrame, servers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the number of servers needed to satisfy the demand for all server generations.

    Args:
        demand_df (pd.DataFrame): DataFrame containing demand data for all server generations.
        servers_info (pd.DataFrame): DataFrame containing server information, including capacity.

    Returns:
        pd.DataFrame: Input DataFrame with additional columns 
                      for the number of servers needed for each generation.
    """
    # List of all server generations
    server_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']

    # Create a dictionary to store server capacities
    server_capacities = servers_df.set_index('server_generation')['capacity'].to_dict()

    for generation in server_generations:
        # Calculate the number of servers needed for each generation
        servers_needed_col = f'{generation}_servers_needed'
        demand_df[servers_needed_col] = (demand_df[generation] / server_capacities[generation]).apply(np.ceil).astype(int)
    return demand_df

In [30]:
demand_with_servers_needed = calculate_servers_needed(demand, servers)
current_demand = demand_with_servers_needed.loc[demand_with_servers_needed['time_step'] == 1]

In [31]:
current_demand

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3,CPU.S1_servers_needed,CPU.S2_servers_needed,CPU.S3_servers_needed,CPU.S4_servers_needed,GPU.S1_servers_needed,GPU.S2_servers_needed,GPU.S3_servers_needed
0,1,high,4000,0,0,0,30,0,0,67,0,0,0,4,0,0
168,1,medium,6000,0,0,0,10,0,0,100,0,0,0,2,0,0
336,1,low,10000,0,0,0,10,0,0,167,0,0,0,2,0,0


In [32]:
for _, latency_demand in current_demand.iterrows():
    print(latency_demand['latency_sensitivity'])
    servers_needed = latency_demand.iloc[9:]
    # servers_to_buy = np.maximum(0, servers_needed - )

high
medium
low


In [33]:
# def calculate_server_profitability(servers: pd.DataFrame,
#                                    selling_prices: pd.DataFrame, 
#                                    datacenters: pd.DataFrame) -> pd.DataFrame:
#     """
#     Calculate the profitability of each server type 
#         based on its revenue and costs, including energy consumption.

#     Args:
#         servers (pd.DataFrame): DataFrame containing server information.
#         selling_prices (pd.DataFrame): DataFrame containing selling price information.
#         datacenters (pd.DataFrame): DataFrame containing datacenter information.

#     Returns:
#         pd.DataFrame: A DataFrame with the original server information plus the
#         following additional columns:
#         - total_revenue (float): Total expected revenue over the server's lifetime.
#         - total_cost (float): Total expected cost over the server's lifetime, including energy costs.
#         - profitability (float): Difference between total_revenue and total_cost.
#     """
#     # Merge server information with selling prices
#     servers_profit = pd.merge(selling_prices, servers, on='server_generation')
    
#     # Calculate average energy cost across all datacenters
#     avg_energy_cost = datacenters['cost_of_energy'].mean()
    
#     # Calculate total revenue
#     servers_profit['total_revenue'] = (
#         servers_profit['capacity'] * 
#         servers_profit['selling_price'] * 
#         servers_profit['life_expectancy']
#     )
    
#     # Calculate total cost, including energy consumption
#     servers_profit['total_cost'] = (
#         servers_profit['purchase_price'] +
#         (servers_profit['average_maintenance_fee'] * servers_profit['life_expectancy']) +
#         (servers_profit['energy_consumption'] * avg_energy_cost * servers_profit['life_expectancy'])
#     )
    
#     # Calculate profitability
#     servers_profit['profitability'] = servers_profit['total_revenue'] - servers_profit['total_cost']
    
#     return servers_profit

In [34]:
# Define all possible latency sensitivities and server generations
all_latencies = ['low', 'medium', 'high']
all_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']

# Create a full grid of all combinations
index = pd.MultiIndex.from_product([all_latencies, all_generations], 
                                    names=['latency_sensitivity', 'server_generation'])

srv_counts = pd.DataFrame(0, index=index, columns=['count']).unstack(level='server_generation', fill_value=0)

In [35]:
srv_counts

count                                          
server_generation   CPU.S1 CPU.S2 CPU.S3 CPU.S4 GPU.S1 GPU.S2 GPU.S3
latency_sensitivity                                                 
high                     0      0      0      0      0      0      0
low                      0      0      0      0      0      0      0
medium                   0      0      0      0      0      0      0

In [36]:
# if not state.fleet.empty:
#         actual_counts = state.fleet.groupby(['latency_sensitivity', 'server_generation']).size().unstack(fill_value=0)
#         srv_counts.update(actual_counts)

In [37]:
demand

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3,CPU.S1_servers_needed,CPU.S2_servers_needed,CPU.S3_servers_needed,CPU.S4_servers_needed,GPU.S1_servers_needed,GPU.S2_servers_needed,GPU.S3_servers_needed
0,1,high,4000,0,0,0,30,0,0,67,0,0,0,4,0,0
1,2,high,8160,0,0,0,61,0,0,136,0,0,0,8,0,0
2,3,high,11016,0,0,0,95,0,0,184,0,0,0,12,0,0
3,4,high,13953,0,0,0,130,0,0,233,0,0,0,17,0,0
4,5,high,19186,0,0,0,168,0,0,320,0,0,0,21,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,164,low,0,0,243279,748553,0,763,4629,0,0,2028,4679,0,96,579
500,165,low,0,0,255443,785981,0,733,4767,0,0,2129,4913,0,92,596
501,166,low,0,0,268216,825280,0,700,4908,0,0,2236,5158,0,88,614
502,167,low,0,0,281626,866544,0,666,5055,0,0,2347,5416,0,84,632


In [149]:
from evaluation import get_actual_demand

In [150]:
actual_demand = get_actual_demand(demand)

latency_sensitivity  time_step server_generation    high      low   medium
0                            1            CPU.S1    4201      721     8998
1                            1            GPU.S1      45        5        4
2                            2            CPU.S1    8166        0    18470
3                            2            GPU.S1     104        9        3
4                            3            CPU.S1    9869     1813    26276
..                         ...               ...     ...      ...      ...
667                        167            GPU.S3    8346     4649     4173
668                        168            CPU.S3  142978   514899    49779
669                        168            CPU.S4  439657  1058682  1084182
670                        168            GPU.S2     610       13      122
671                        168            GPU.S3    8050     5061     3891

[672 rows x 5 columns]


In [151]:
actual_demand

latency_sensitivity,time_step,server_generation,high,low,medium
0,1,CPU.S1,4201,721,8998
1,1,GPU.S1,45,5,4
2,2,CPU.S1,8166,0,18470
3,2,GPU.S1,104,9,3
4,3,CPU.S1,9869,1813,26276
...,...,...,...,...,...
667,167,GPU.S3,8346,4649,4173
668,168,CPU.S3,142978,514899,49779
669,168,CPU.S4,439657,1058682,1084182
670,168,GPU.S2,610,13,122


In [152]:
def calculate_servers_needed(demand_df: pd.DataFrame, servers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the number of servers needed to satisfy the demand for all server generations.
    Args:
        demand_df (pd.DataFrame): DataFrame containing demand data for all server generations.
        servers_df (pd.DataFrame): DataFrame containing server information, including capacity.
    Returns:
        pd.DataFrame: Input DataFrame with number of servers needed instead of capacity
    Note:
        The function does not take into account the current fleet. 
        It is intended to be ran once, before the time steps loop.
    """
    # Merge demand_df with servers_df to get capacity information
    merged_df = pd.merge(demand_df, servers_df[['server_generation', 'capacity']], on='server_generation', how='left')
    
    # Calculate servers needed for each demand scenario
    for scenario in ['high', 'medium', 'low']:
        merged_df[f'{scenario}_servers_needed'] = np.ceil(merged_df[scenario] / merged_df['capacity']).astype(int)
    
    # Drop the capacity column and rename the new columns
    result_df = merged_df.drop(columns=['capacity', 'high', 'medium', 'low'])
    result_df = result_df.rename(columns={
        'high_servers_needed': 'high',
        'medium_servers_needed': 'medium',
        'low_servers_needed': 'low'
    })
    
    return result_df


In [154]:
result = calculate_servers_needed(actual_demand, servers)
x = result.loc[result['time_step'] == 1]
x

,time_step,server_generation,high,medium,low
0,1,CPU.S1,71,150,13
1,1,GPU.S1,6,1,1


In [84]:
def summarize_fleet_deployment(fleet_df: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize the fleet deployment by server generation and datacenter latency.
    
    Args:
        fleet_df (pd.DataFrame): DataFrame containing fleet information.
    
    Returns:
        pd.DataFrame: Summary of servers deployed by generation and latency sensitivity.
    """
    # Group by server generation and latency sensitivity, then count the servers
    summary = fleet_df.groupby(['server_generation', 'latency_sensitivity']).size().unstack(fill_value=0)
    
    # Ensure all required columns are present, add them with zeros if missing
    for col in ['high', 'medium', 'low']:
        if col not in summary.columns:
            summary[col] = 0
    
    # Select only the required columns and reset the index
    summary = summary[['high', 'medium', 'low']].reset_index()
    
    # Sort the DataFrame by server generation
    summary = summary.sort_values('server_generation')
    
    return summary


In [85]:
# Example usage:
fleet_data = {
    'datacenter_id': ['DC1', 'DC2', 'DC3', 'DC1', 'DC2'],
    'server_generation': ['CPU.S1', 'CPU.S2', 'CPU.S4', 'GPU.S2', 'GPU.S3'],
    'server_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
    'lifespan': [10, 15, 20, 25, 30],
    'life_expectancy': [96, 96, 96, 96, 96],
    'latency_sensitivity': ['low', 'medium', 'high', 'low', 'medium'],
    'capacity': [75, 120, 160, 8, 8],
    'purchase_price': [16000, 19500, 22000, 140000, 160000],
    'average_maintenance_fee': [308, 375, 423, 2695, 3080],
    'energy_consumption': [460, 800, 920, 3000, 4200],
    'cost_of_energy': [100, 100, 100, 100, 100],
    'moved': [0, 0, 0, 0, 0]
}

fleet_df = pd.DataFrame(fleet_data)

result = summarize_fleet_deployment(fleet_df)
print(result)

latency_sensitivity server_generation  high  medium  low
0                              CPU.S1     0       0    1
1                              CPU.S2     0       1    0
2                              CPU.S4     1       0    0
3                              GPU.S2     0       0    1
4                              GPU.S3     0       1    0


In [103]:
def calculate_demand_satisfaction(fleet_summary: pd.DataFrame, demand_in_servers: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the difference between deployed servers and demand for each server generation and latency.
    
    Args:
        fleet_summary (pd.DataFrame): Summary of deployed servers by generation and latency.
        demand_in_servers (pd.DataFrame): Demand for servers by generation and latency.
    
    Returns:
        pd.DataFrame: Difference between deployed servers and demand. Positive values indicate excess capacity,
                      negative values indicate unmet demand.
    """
    # Step 1: Create a dataframe with all server generations and initialize with zeros
    all_generations = ['CPU.S1', 'CPU.S2', 'CPU.S3', 'CPU.S4', 'GPU.S1', 'GPU.S2', 'GPU.S3']
    result = pd.DataFrame({
        'server_generation': all_generations,
        'high': 0,
        'medium': 0,
        'low': 0
    })
    
    # Step 2: Update the result with fleet summary data
    for _, row in fleet_summary.iterrows():
        gen = row['server_generation']

        if gen in result['server_generation'].values:
            mask = result['server_generation'] == gen

            for col in ['high', 'medium', 'low']:
                result.loc[mask, col] = row[col]
    
    # Step 3: Subtract demand from the result
    for _, row in demand_in_servers.iterrows():
        gen = row['server_generation']
        
        if gen in result['server_generation'].values:
            mask = result['server_generation'] == gen
            
            for col in ['high', 'medium', 'low']:
                result.loc[mask, col] = result.loc[mask, col].values - row[col]
    
    return result


In [87]:
demand_sat = calculate_demand_satisfaction(result, x)
demand_sat

,server_generation,high,medium,low
0,CPU.S1,-1336,-3178,-7683
1,CPU.S2,-1785,-1656,-5123
2,CPU.S3,0,0,0
3,CPU.S4,1,0,0
4,GPU.S1,-527,-95,-118
5,GPU.S2,-2,-5,0
6,GPU.S3,0,1,0


In [88]:
def move_servers_to_satisfy_demand(fleet_df: pd.DataFrame, 
                                   calculated_demand_satisfaction: pd.DataFrame, 
                                   datacenters_df: pd.DataFrame) -> list:
    """
    Move servers between datacenters to satisfy demand based on calculated demand satisfaction.
    
    Args:
        fleet_df (pd.DataFrame): DataFrame containing current fleet information.
        calculated_demand_satisfaction (pd.DataFrame): DataFrame with demand satisfaction calculations.
        datacenters_df (pd.DataFrame): DataFrame with datacenter information.
    
    Returns:
        list: List of dictionaries containing server movement actions.
    """
    actions = []
    LATENCY_TO_DC = {
        'low': 'DC1',
        'medium': 'DC2',
        'high': ['DC3', 'DC4']
    }
    
    for _, row in calculated_demand_satisfaction.iterrows():
        server_generation = row['server_generation']
        for from_latency in ['high', 'medium', 'low']:
            excess = row[from_latency]
            if excess <= 0:
                continue
            
            for to_latency in ['high', 'medium', 'low']:
                deficit = -row[to_latency]
                if deficit <= 0:
                    continue
                
                servers_to_move = min(excess, deficit)
                
                # Find servers to move
                servers = fleet_df[(fleet_df['server_generation'] == server_generation) & 
                                   (fleet_df['datacenter_id'].isin([LATENCY_TO_DC[from_latency]] if isinstance(LATENCY_TO_DC[from_latency], str) else LATENCY_TO_DC[from_latency]))]
                
                for _, server in servers.iterrows():
                    if servers_to_move == 0:
                        break
                    
                    # Determine target datacenter
                    if to_latency == 'high':
                        target_dcs = LATENCY_TO_DC[to_latency]
                        for dc in target_dcs:
                            dc_info = datacenters_df[datacenters_df['datacenter_id'] == dc].iloc[0]
                            if dc_info['slots_capacity'] > dc_info['used_slots']:
                                target_dc = dc
                                break
                        else:
                            continue  # No space in high latency datacenters
                    else:
                        target_dc = LATENCY_TO_DC[to_latency]
                        dc_info = datacenters_df[datacenters_df['datacenter_id'] == target_dc].iloc[0]
                        if dc_info['slots_capacity'] <= dc_info['used_slots']:
                            continue  # No space in target datacenter
                    
                    # Move server
                    actions.append({
                        'datacenter_id': target_dc,
                        'server_generation': server_generation,
                        'server_id': server['server_id'],
                        'action': 'move'
                    })
                    
                    # Update datacenter used slots
                    datacenters_df.loc[datacenters_df['datacenter_id'] == target_dc, 'used_slots'] += 1
                    
                    servers_to_move -= 1
                    excess -= 1
                    row[from_latency] -= 1
                    row[to_latency] += 1
                
                if excess == 0:
                    break
            
            if excess == 0:
                break
    
    return actions

In [159]:
def move_servers_to_satisfy_demand(fleet_df: pd.DataFrame, 
                                   calculated_demand_satisfaction: pd.DataFrame, 
                                   datacenters_df: pd.DataFrame) -> list:
    """
    Move servers between datacenters to satisfy demand based on calculated demand satisfaction.
    
    Args:
        fleet_df (pd.DataFrame): DataFrame containing current fleet information.
        calculated_demand_satisfaction (pd.DataFrame): DataFrame with demand satisfaction calculations.
        datacenters_df (pd.DataFrame): DataFrame with datacenter information.
    
    Returns:
        list: List of dictionaries containing server movement actions.
    """
    actions = []
    LATENCY_TO_DC = {
        'low': 'DC1',
        'medium': 'DC2',
        'high': ['DC3', 'DC4']
    }

    updated_demand_satisfaction = calculated_demand_satisfaction.copy()
    
    for idx, row in calculated_demand_satisfaction.iterrows():
        server_generation = row['server_generation']
        print(server_generation)
        for from_latency in ['high', 'medium', 'low']:
            excess = row[from_latency]
            if excess <= 0:
                print('excess')
                continue
            
            for to_latency in ['high', 'medium', 'low']:
                deficit = -row[to_latency]
                if deficit <= 0:
                    print('deficit')

                    continue
                
                servers_to_move = min(excess, deficit)
                
                # Find servers to move
                servers = fleet_df[(fleet_df['server_generation'] == server_generation) & 
                                   (fleet_df['datacenter_id'].isin([LATENCY_TO_DC[from_latency]] if isinstance(LATENCY_TO_DC[from_latency], str) else LATENCY_TO_DC[from_latency]))]
                
                for _, server in servers.iterrows():
                    if servers_to_move == 0:
                        break
                    
                    # Determine target datacenter
                    if to_latency == 'high':
                        target_dcs = LATENCY_TO_DC[to_latency]
                        for dc in target_dcs:
                            dc_info = datacenters_df[datacenters_df['datacenter_id'] == dc].iloc[0]
                            if dc_info['slots_capacity'] > dc_info['used_slots']:
                                target_dc = dc
                                break
                        else:
                            continue  # No space in high latency datacenters
                    else:
                        target_dc = LATENCY_TO_DC[to_latency]
                        dc_info = datacenters_df[datacenters_df['datacenter_id'] == target_dc].iloc[0]
                        if dc_info['slots_capacity'] <= dc_info['used_slots']:
                            continue  # No space in target datacenter
                    
                    # Move server
                    actions.append({
                        'datacenter_id': target_dc,
                        'server_generation': server_generation,
                        'server_id': server['server_id'],
                        'action': 'move'
                    })
                    
                    # Update datacenter used slots
                    datacenters_df.loc[datacenters_df['datacenter_id'] == target_dc, 'used_slots'] += 1
                    
                    servers_to_move -= 1
                    excess -= 1

                    updated_demand_satisfaction.at[idx, from_latency] -= 1
                    updated_demand_satisfaction.at[idx, to_latency] += 1
                
                if excess == 0:
                    break
            
            if excess == 0:
                break
    
    return actions, updated_demand_satisfaction

In [143]:
def buy_servers_to_satisfy_demand(calculated_demand_satisfaction: pd.DataFrame, 
                                  actions: list) -> list:
    """
    Determine if new servers need to be bought to satisfy remaining demand 
        and add 'buy' actions if necessary.
    
    Args:
        calculated_demand_satisfaction (pd.DataFrame): DataFrame with 
                   current demand satisfaction calculations.
        actions (list): List of existing actions (moves, etc.).
    
    Returns:
        list: Updated list of actions, including new 'buy' actions if needed.
    """
    LATENCY_TO_DC = {
        'low': 'DC1',
        'medium': 'DC2',
        'high': 'DC3'  # We'll use DC3 for high latency purchases
    }
    
    for _, row in calculated_demand_satisfaction.iterrows():
        server_generation = row['server_generation']
        for latency in ['high', 'medium', 'low']:
            deficit = max(0, -row[latency])  # Negative values indicate need for more servers
            if deficit > 0:
                print(deficit)
                target_dc = LATENCY_TO_DC[latency]
                for i in range(deficit):
                    print(i)

                    actions.append({
                        'datacenter_id': target_dc,
                        'server_generation': server_generation,
                        'server_id': str(uuid.uuid4()),
                        'action': 'buy'
                    })
    
    return actions

In [109]:
fleet_test = pd.read_csv('fleet_test.csv')
demand_test = pd.read_csv('demand_test.csv')
dc_test = pd.read_csv('dc_test.csv')
# count_servers = pd.read_csv('count_servers.csv')
calculated_demand_satisfaction = pd.read_csv('calculated_demand_satisfaction.csv')

In [102]:
fleet_sum_test = summarize_fleet_deployment(fleet_test)
fleet_sum_test

latency_sensitivity,server_generation,high,medium,low
0,CPU.S2,0,4,0
1,GPU.S3,6,0,0


In [106]:
calculated_demand_satisfaction_test = calculate_demand_satisfaction(fleet_sum_test, demand_test)

In [160]:
actions, new_sat = move_servers_to_satisfy_demand(fleet_test, calculated_demand_satisfaction, dc_test)

CPU.S1
excess
excess
excess
CPU.S2
excess
CPU.S3
excess
excess
excess
CPU.S4
excess
excess
excess
GPU.S1
excess
excess
excess
GPU.S2
excess
excess
excess
GPU.S3
deficit
deficit
excess
excess


In [145]:
actions_with_buy = buy_servers_to_satisfy_demand(new_sat, actions)

1
0


In [161]:
actions_with_buy

[{'datacenter_id': 'DC3',
  'server_generation': 'CPU.S2',
  'server_id': '3dffc8ab-edac-4fdb-a3d6-878c7c323f7c',
  'action': 'move'},
 {'datacenter_id': 'DC3',
  'server_generation': 'CPU.S2',
  'server_id': '36196cf0-b022-4707-9f33-eac4163d1510',
  'action': 'move'},
 {'datacenter_id': 'DC3',
  'server_generation': 'CPU.S2',
  'server_id': 'cd10a497-171c-4bd1-9f25-b2d4754b4301',
  'action': 'move'},
 {'datacenter_id': 'DC3',
  'server_generation': 'CPU.S2',
  'server_id': '236ba91a-d1d8-438b-b9f2-4d65ad25cfdb',
  'action': 'move'},
 {'datacenter_id': 'DC1',
  'server_generation': 'GPU.S3',
  'server_id': '0ee612e6-8e1e-494b-935a-cfc6cafe69bf',
  'action': 'move'},
 {'datacenter_id': 'DC1',
  'server_generation': 'GPU.S3',
  'server_id': '80e90d2c-6a26-4725-a63e-67d63f4cacca',
  'action': 'move'},
 {'datacenter_id': 'DC1',
  'server_generation': 'GPU.S3',
  'server_id': '41c83005-739a-431e-8f22-db37fcdf0dcb',
  'action': 'move'},
 {'datacenter_id': 'DC3',
  'server_generation': 'CPU.S

In [156]:
from system_state import SystemState

state = SystemState(datacenters, servers)
state.servers_info.loc[state.servers_info['server_generation'] == 'CPU.S1'].iloc[0]['slots_size']

2